# SOCIAL ANXIETY PREVENTION RECOMMENDER

## PROBLEM STATEMENT 

With over 5 million Kenyans struggling with anxiety disorders, MindGuard revolutionizes mental health through an intelligent early-warning system. Using Feedforward Neural Networks to interpret complex behavioral patterns, MindGuard detects anxiety risks and delivers personalized preventative interventions, helping individuals maintain optimal mental wellness before clinical intervention becomes necessary.

## OBJECTIVES

1. Can we create an intelligent early-warning system that accurately predicts who is likely to experience anxiety problems, allowing for timely prevention and support?

2. How can we provide personalized lifestyle recommendations that are most likely to help individuals reduce their anxiety risk based on their unique profile?

3. Which lifestyle factors have the biggest impact on anxiety and how can these insights guide effective prevention strategies and public health initiatives?




## PLAN OF EXECUTION

- EDA 
- PREPROCESSING
- MODEL BUILDING
- MODEL INTERPRETATION 
- DEPLOYMENT



## EDA AND PREPROCESSING

### Load Dataset

In [ ]:
#import needed libraries and load datasets to be used
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


enhanced_data = pd.read_csv('enhanced_anxiety_dataset.csv')
family_data = pd.read_csv('family_anxiety_14_dataset.csv')



: 

### Perusing Data


Glancing at the dataset and what it entails to help guide on how we will execute our objectives

In [ ]:
#Print first rows of enhanced anxiety dataset
enhanced_data.head()


In [ ]:
enhanced_data.columns

In [ ]:
#Scheme the data types of the enhanced dataset
enhanced_data.info()

In [ ]:
#Summary stats for enhanced dataset
enhanced_data.describe()

In [ ]:
#Check for missing values
enhanced_data.isnull().sum()

### Preprocessing pipelines and Modeling using ensemble methods 

This defined broke down the features into 2 Feature categories. We then proceeded to carrying out Preprocessing and Modeling using Random Forest based on the Objectives of the project

In [ ]:
#Define feature categories
ls_features = ['Age', 'Gender', 'Occupation',
    'Sleep Hours', 'Physical Activity (hrs/week)', 
    'Caffeine Intake (mg/day)', 'Alcohol Consumption (drinks/week)',
    'Smoking', 'Family History of Anxiety', 'Stress Level (1-10)',  'Recent Major Life Event',
    'Diet Quality (1-10)'
]
physiological_features = ['Heart Rate (bpm)', 'Breathing Rate (breaths/min)', 'Sweating Level (1-5)','Dizziness']
  
all_features = ls_features + physiological_features

In [ ]:
enhanced_data[all_features].info()

In [ ]:
#Split data into features and target variable
anxiety_thresh = 5
X_lifestyle= enhanced_data[ls_features].copy()
X = enhanced_data[all_features].copy()

y_bin = (enhanced_data['Anxiety Level (1-10)'] >= anxiety_thresh).astype(int)
y = enhanced_data['Anxiety Level (1-10)'].values



In [ ]:
X.info()

In [ ]:
#Identify Categorical and numerical features
categorical_features = X.select_dtypes(include=['object']).columns
num_features = X.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
print("Numerical transformer:", num_features.tolist())
print("Categorical transformer:", categorical_features.tolist())

In [ ]:
# Define preprocessing steps for numerical and categorical features


early_preprocessing = ColumnTransformer(
    transformers=[ ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
)


In [ ]:
#Split the data into training and testing sets
X_train_rf,X_test_rf,y_train_bin_rf,y_test_bin_rf = train_test_split(X_lifestyle,y_bin, test_size=0.2, random_state=42, stratify=y_bin)


In [ ]:

X_train_all,X_test_all,y_train_all,y_test_all = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
print("Available columns in X_train_rf:")
print(X_train_rf.columns.tolist())

In [ ]:
# Objective 1: Early Detection of Anxiety using Random Forest Classifier
# Create a pipeline with preprocessing and classifier

# Objective 1: Early Detection of Anxiety using Random Forest Classifier
# Create a preprocessing pipeline for lifestyle features only
lifestyle_num_features = X_train_rf.select_dtypes(include=['int64', 'float64']).columns
lifestyle_cat_features = X_train_rf.select_dtypes(include=['object']).columns

lifestyle_preprocessing = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), lifestyle_num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), lifestyle_cat_features)
    ]
)

early_pip = Pipeline([
    ('preprocessing', lifestyle_preprocessing),
    ('classifier', RandomForestClassifier(n_estimators=200, 
     max_depth=15, min_samples_split=5, random_state=42, class_weight='balanced'))
])

early_pip.fit(X_train_rf, y_train_bin_rf)
early_pred = early_pip.predict(X_test_rf)
early_acc = accuracy_score(y_test_bin_rf, early_pred)

In [ ]:
print(f"RF Early Detection Accuracy: {early_acc: .4f}")
print(classification_report(y_test_bin_rf,early_pred, target_names=['Low Anxiety', 'High Anxiety']))

In [ ]:

# Create a pipeline with preprocessing and classifier
rf_rec_pipeline = Pipeline([
	('preprocessing', early_preprocessing),
	('classifier', RandomForestRegressor(random_state=42, n_estimators=200, max_depth=20, min_samples_split=2))
])


In [ ]:
#Objective 2: Personalized recommendations for anxiety management
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

#Fit the pipeline to the training data
rf_rec_pipeline.fit(X_train_all,y_train_all)

#Test predictions and evaluate the model
y_pred = rf_rec_pipeline.predict(X_test_all)

#Evaluate model performance
rf_rmse = np.sqrt(mean_squared_error(y_test_all, y_pred))
r2 = r2_score(y_test_all, y_pred)

In [ ]:
print(f"RF Regression RMSE: {rf_rmse: .4f}")
print(f"RF Regression R^2: {r2: .4f}")

In [ ]:
#Objective 3: Lifestyle Analysis
rf_feature_names = (list(num_features) + list(early_preprocessing.named_transformers_['cat'].get_feature_names_out(categorical_features)))

In [ ]:
rf_importance = rf_rec_pipeline.named_steps['classifier'].feature_importances_

rf_importance_df = pd.DataFrame({'feature' : rf_feature_names,
                                 'importance': rf_importance}).sort_values('importance', ascending=False)

In [ ]:
print("Top 10 RF Feature Importances:")
print(rf_importance_df.head(10))


In [ ]:
#Visualize feature importances
fig, ax = plt.subplots(figsize=(10, 8))
axes = [ax]

rf_importance_df.head(10).plot(kind='barh', x='feature', y='importance', ax=axes[0], legend=False)




In [ ]:
#Random Forest Performance Summary
rf_metrics = ['Early Warning\nAccuracy', 'Regression\nRMSE', 'Regression\nR²']
rf_values = [early_acc, rf_rmse, r2]
colors = ['lightblue', 'lightcoral', 'lightgreen']

In [ ]:
fig_perf, ax_perf = plt.subplots(figsize=(6, 5))
bars = ax_perf.bar(rf_metrics, rf_values, color=colors)
ax_perf.set_title('Random Forest Performance Summary')
ax_perf.set_ylabel('Score')
plt.show()

In [ ]:
#Performance summary
print(f"RF Early Detection Accuracy: {early_acc: .4f}")
print(classification_report(y_test_bin_rf,early_pred, target_names=['Low Anxiety', 'High Anxiety']))
print("Top 10 RF Feature Importances:")
print(rf_importance_df.head(10))



## MODEL BUILDING 


In [ ]:
# Import modeling and evaluation libraries

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

In [ ]:
# Load and combine raw datasets
enhanced_df = pd.read_csv('enhanced_anxiety_dataset.csv')
family_df = pd.read_csv('family_anxiety_14_dataset.csv')

# Label dataset origin
enhanced_df['Dataset'] = 'Enhanced'
family_df['Dataset'] = 'Family'

# Combine datasets
data = pd.concat([enhanced_df, family_df], ignore_index=True)
data.head()

# Encode categorical variables
categorical_cols = ['Gender', 'Occupation', 'Smoking', 'Family History of Anxiety', 'Dizziness', 'Medication', 'Recent Major Life Event', 'Dataset']
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# Handle missing values (if any)
data_encoded = data_encoded.dropna()

# Define features and target
X = data_encoded.drop('Anxiety Level (1-10)', axis=1)
y = data_encoded['Anxiety Level (1-10)']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Baseline Model

In [ ]:
# Baseline Model: Linear Regression
# Simple baseline to set a performance floor

from sklearn.linear_model import LinearRegression
baseline = LinearRegression()
baseline.fit(X_train, y_train)
y_pred_baseline = baseline.predict(X_test)

print("Baseline Linear Regression")
print(f"MSE: {mean_squared_error(y_test_all, y_pred_baseline):.2f}, R^2: {r2_score(y_test, y_pred_baseline):.2f}")


### Advanced Models

In [ ]:
# Define a suite of regression models for comparison
# This gives a comparative view of multiple modeling approaches 
# from linear to tree-based to neural helping identify the model that best captures patterns in the data.

models = {
    'Ridge Regression': Ridge(alpha=1.0),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'Feedforward Neural Net': MLPRegressor(hidden_layer_sizes=(128, 64, 32), 
                                           activation='relu',
                                           solver='adam',
                                           max_iter=1000,
                                           random_state=42)
}

# Train each model and evaluate on the test set

results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results.append((name, mse, r2))

# Results DataFrame 
results_df = pd.DataFrame(results, columns=["Model", "MSE", "R2"])
results_df.sort_values(by="R2", ascending=False)

#### Evaluation of Models

Random Forest performed best with MSE = 1.07 and R² = 0.77, making it the most accurate and generalizable model.

Gradient Boosting and the Feedforward Neural Network followed closely, also explaining over 75% of the variance.

Ridge Regression and the baseline Linear model showed limited predictive power (R² ≈ 0.70), suggesting linear models miss important patterns.

Decision Tree underperformed, indicating it's too simple for this task.

## Feedforward Neural Network
### Why?
Strong performance: R² of 0.75, close to the best-performing model.

Captures complex patterns: Better at learning nonlinear relationships among lifestyle and physiological factors.

Future-ready: Easily extendable for classification, personalization, and deep learning deployment.

Strategic fit: Aligns with MindGuard’s AI-first approach for real-time, intelligent anxiety risk prediction.



### Predicted vs Actual

In [ ]:
import seaborn as sns

fnn_model = models['Feedforward Neural Net']
y_pred_nn = fnn_model.predict(X_test)

plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_pred_nn, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual Anxiety Level")
plt.ylabel("Predicted Anxiety Level")
plt.title("FNN: Predicted vs Actual")
plt.tight_layout()
plt.show()


Observations: 

1. Tight clustering around the red line shows strong agreement between predicted and actual anxiety levels.
2. Spread is consistent across the range, meaning the model performs well across both low and high anxiety scores.
3. Minor vertical scatter indicates some prediction error, but no major bias or drift.
4. Overall, the FNN demonstrates accurate and stable predictions across the full anxiety scale.

### Residual Plot

In [ ]:
residuals = y_test - y_pred_nn

plt.figure(figsize=(8, 6))
sns.histplot(residuals, bins=30, kde=True)
plt.title("FNN Residuals Distribution")
plt.xlabel("Residual (Actual - Predicted)")
plt.tight_layout()
plt.show()


Residual Plot Summary:

1. Symmetrical bell shape: The residuals are normally distributed meaning the model makes balanced predictions with no major bias.
2. Centered at 0: On average, the model neither overpredicts nor underpredicts which is the ideal behavior.
3. Tapering ends: Most predictions are close to the actual values; very large errors are rare.
4. Smooth KDE line	The kernel density estimation (blue line) follows the histogram closely, confirming stable prediction behavior across data points.

## MODEL INTERPRETATION

### Early detection of Anxiety using Random Forest Classifier
#### Interpretations 
High Accuracy (82%): Indicates the model successfully distinguishes between individuals with low and high anxiety.

Precision (Low Anxiety) = 0.87: The model is very reliable when it predicts someone does not have high anxiety—good for avoiding false alarms.

Recall (High Anxiety) = 0.70: About 70% of those with high anxiety are correctly identified, which is decent but leaves room for improvement.

F1-score (High Anxiety) = 0.71: Balanced performance in predicting high-risk individuals.

#### Insights
Slight Recall Imbalance: Could indicate:

Class imbalance (fewer high-anxiety cases).

Need for SMOTE or weighted loss.

Random Forest Strength: Works well with non-linearities and feature interactions—ideal for psychological data with complex dependencies.

Feature Contribution: Lifestyle data alone allows fairly strong early detection, emphasizing the value of behavioral data for screening purposes.
### Personalized recommendations(regression)
#### Key Model Interpretations:
R² = 0.77: The model explains 77% of the variance in anxiety scores across individuals. This is a strong indicator of model generalization.

RMSE = 1.03: On a 1–10 scale, the average prediction error is around 1 point—acceptable for psychological risk prediction.
#### Insights
Non-Linearity Captured: Anxiety is not always linearly related to factors like age or caffeine. RF handles this with ease, making predictions robust.

Feature Flexibility: By combining lifestyle and physiological data, the model gives a more holistic view of each individual, enabling nuanced recommendations.

Personalization: You can reverse-engineer the score and provide users with:

“You are scoring 7.2 due to high stress and low sleep.”

“Improving sleep from 4h to 7h could reduce your risk score by ~1.1.”
### Lifestyle Analysis: Feature Importance
#### Interpretations
Stress Level (0.47): Core driver of anxiety — confirms psychological theory. Strongest single predictor in both classifier and regressor.

Sleep Hours (0.22): Inadequate sleep strongly elevates anxiety. Tailored sleep hygiene interventions can be highly effective.

Caffeine Intake (0.08): Stimulants like caffeine exacerbate anxiety—important for behavioral recommendations.

Diet Quality (0.04): Poor nutrition subtly contributes to anxiety—likely via energy, gut-brain axis, or inflammation.

#### Insights
Interventions can be personalized based on individual feature profiles:

High caffeine + low sleep = focus on stimulants and rest.

High stress + poor breathing = recommend mindfulness, yoga, or CBT.

Explainable AI Opportunity: With SHAP values, you can show users exactly which features pushed their score up/down—enhancing trust and transparency.

### Interpretations across all models.
Model Agreement
All ensemble models (Random Forest, Gradient Boosting) perform well, suggesting:
The signal in the data is strong and consistent.
Non-linear interactions dominate (e.g., stress x sleep, age x caffeine).

FNN also performs well (R² = 0.75), confirming deep learning can learn useful patterns even from moderate-sized tabular data.

Model Robustness
Random Forest has low variance and bias due to its averaging approach.

The regressor’s consistent performance across test sets suggests it can be safely deployed for real-world prediction (if retrained on production data).

Health Insight Integration
The model’s importance rankings correlate with clinical/psychological literature—validating its practical utility and trustworthiness.

## CONCLUSION 

This project successfully demonstrates the potential of machine learning models—specifically ensemble methods like Random Forest and Gradient Boosting—in the early detection, risk prediction, and personalized recommendation of anxiety prevention strategies. 

By leveraging a combination of lifestyle and physiological data, we have built interpretable, high-performing models capable of supporting real-world mental health interventions.

This work demonstrates that machine learning can meaningfully contribute to mental health awareness and early intervention. The models developed here can be embedded into digital health tools to provide users with:

Early warning signs of potential anxiety disorders.

Risk scores tailored to their personal lifestyle and physiology.

Recommendations for behavioral change targeting high-impact factors such as stress management, sleep hygiene, and caffeine reduction.

## NEXT STEPS